In [1]:
import numpy as np
import helpers as h
from implementations import *

In [2]:
X, xHeader, Y, yHeader, indexedX, indexedXheader, indexedY, indexedYheader = loadTrainingData()

Data successfully loaded, there are 321 features and 328135 samples, the shapes of the unindexed data is:
y: (328135, 1), x: (328135, 321)


In [3]:
yClean, xClean, xHeaderClean = dataCleaning(Y,X,xHeader)

For a threshold of 0.7, there are 144 good features, and 177 bad features
There remains in the data 289870 samples with at most 5 missing values
The number of invalid entries remaing in the dataset is 272938
That is 0.006538802834987332 parts of the whole dataset
Removed 21050 samples with outliers more than 10 standard deviations from the mean. There remains 268820 samples in the dataset.
Standardized data by subtracting the mean and dividing by the standard deviation


In [4]:
yBalanced, xBalanced = balanceData(yClean,xClean)
tx = makeTrainingData(xBalanced)

Created a balanced subset of the data, with 46448 samples, 23224 each of positive and negative samples
Added dummy variable and replaced invalid entries with zeros


In [8]:
initial_w = np.zeros(tx.shape[1])
w_gd_m, loss_gd_m = mse_gd_momentum(yBalanced,tx,initial_w,1000,0.1)

In [9]:
print(compute_loss(yBalanced,tx,initial_w))
print(loss_gd_m)
print(logistic_loss(yBalanced,tx,w_gd_m))


1.0
0.6214366735917236
25530.600562960608


In [10]:
w_logistic, loss_logistic = logistic_regression(yBalanced,tx,initial_w,100,0.01)

#print(compute_loss(yBalanced,tx,initial_w))
#print(compute_loss(yBalanced,tx,w_logistic))

print(logistic_loss(yBalanced,tx,initial_w))
print(logistic_loss(yBalanced,tx,w_logistic))

32195.30024264834
23013.129489674684


In [11]:
print(tx@w_logistic)

[-2.82729983 -1.34065062  1.34702327 ...  1.68714486 -1.54787826
 -1.93685045]
